In [ ]:
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from keybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive

In [ ]:
def clean_text(text):
    """Clean text by removing special characters and extra whitespace"""
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Remove extra whitespace
        text = ' '.join(text.split())
        return text
    return ''


In [ ]:
def extract_keywords_for_row(row, kw_model, num_keywords=5):
    """Extract keywords for a single row"""
    # Combine text fields
    combined_text = f"{row['title']} {row['description']} {row['body']}"
    cleaned_text = clean_text(combined_text)

    if not cleaned_text:
        return [], []

    # Extract keywords
    keywords = kw_model.extract_keywords(
        cleaned_text,
        keyphrase_ngram_range=(1, 2),
        stop_words='english',
        use_maxsum=True,
        nr_candidates=20,
        top_n=num_keywords
    )

    # Separate keywords and scores
    if keywords:
        kw, scores = zip(*keywords)
        return list(kw), list(scores)
    return [], []

In [ ]:
def process_dataset(file_path, num_keywords=5):
    """Process entire dataset and add keyword columns"""
    # Read CSV file
    print("Loading dataset...")
    df = pd.read_csv(file_path)

    # Initialize KeyBERT
    print("Initializing KeyBERT model...")
    kw_model = KeyBERT(model='all-MiniLM-L6-v2')

    # Initialize lists to store keywords and scores
    all_keywords = []
    all_scores = []

    # Process each row
    print("Extracting keywords for each row...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        keywords, scores = extract_keywords_for_row(row, kw_model, num_keywords)
        all_keywords.append(keywords)
        all_scores.append(scores)

    # Add new columns to dataframe
    df['keywords'] = all_keywords
    df['keyword_scores'] = all_scores

    # Add individual keyword columns for easier filtering
    for i in range(num_keywords):
        df[f'keyword_{i+1}'] = df['keywords'].apply(lambda x: x[i] if i < len(x) else '')
        df[f'keyword_{i+1}_score'] = df['keyword_scores'].apply(lambda x: x[i] if i < len(x) else 0.0)

    # Save results
    output_file = 'participedia_with_keywords.csv'
    df.to_csv(output_file, index=False)
    print(f"\nResults saved to {output_file}")

    return df

In [ ]:
def display_sample_results(df, num_samples=5):
    """Display sample results from the processed dataset"""
    print("\n=== Sample Results ===")
    for i in range(min(num_samples, len(df))):
        print(f"\nDocument {i+1}:")
        print(f"Title: {df['title'].iloc[i][:100]}...")
        print("Keywords:", df['keywords'].iloc[i])
        print("Scores:", [f"{score:.3f}" for score in df['keyword_scores'].iloc[i]])
        print("-" * 80)


In [ ]:
def main():
    # Process dataset
    df = process_dataset('/content/drive/MyDrive/Case_Participedia.csv')

    # Display sample results
    display_sample_results(df)

    # Print column information
    print("\nNew columns added to the dataset:")
    keyword_cols = [col for col in df.columns if 'keyword' in col.lower()]
    for col in keyword_cols:
        print(f"- {col}")

if __name__ == "__main__":
    main()

Loading dataset...
Initializing KeyBERT model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Extracting keywords for each row...


100%|██████████| 2272/2272 [20:15<00:00,  1.87it/s]



Results saved to participedia_with_keywords.csv

=== Sample Results ===

Document 1:
Title: British Columbia Citizens' Assembly on Electoral Reform...
Keywords: ['electoral alternatives', 'columbia voters', 'elected provincial', 'analyzing electoral', 'columbia referendum']
Scores: ['0.536', '0.538', '0.556', '0.561', '0.621']
--------------------------------------------------------------------------------

Document 2:
Title: Minneapolis Neighborhood Revitalization Program...
Keywords: ['legislature minneapolis', 'implementation downtown', 'neighborhood nrps', 'improve citys', 'residents revitalize']
Scores: ['0.514', '0.525', '0.527', '0.537', '0.547']
--------------------------------------------------------------------------------

Document 3:
Title: Wenling City Deliberative Poll...
Keywords: ['democracyreferences', 'municipalities china', 'debate deliberation', 'chinese deliberative', 'polling project']
Scores: ['0.480', '0.481', '0.481', '0.486', '0.488']
------------------------

embedding
if two have similar embedding, most likely they have the same topic

given this quary, what filter to apply on the data

reg system. embed the quary embedding


go through natural language to